# Text Classification Exam

Welcome to the Text Classification Practical Exam. In this exam, you will be tasked with building, training, and evaluating an NLP model to classify text data. You are provided with a labeled dataset containing both the text and its corresponding class labels.

Your objective is to develop a model that accurately predicts the class of the given text. Make sure to follow best practices in data preprocessing, model selection, and evaluation to achieve optimal results.

Good luck!
___

# Install and Import Needed Libraries

In [2]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.9 MB/s eta 0:00:00


In [22]:
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D
import string
import contractions
import keras

# Download the Dataset

In [4]:
!kaggle datasets download -d khaledzsa/spam-messages
!unzip spam-messages.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/spam-messages
License(s): unknown
  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 66.1MB/s]
Archive:  spam-messages.zip
  inflating: spam.csv                


In [96]:
df = pd.read_csv('spam.csv')

# Data Exploration

Before diving into preprocessing and model building, it’s important to first explore the dataset to understand its structure, distribution, and key characteristics. This step will help you gain insights into the data and guide your decisions in subsequent steps. Here’s what to consider:

1. **Inspect the Data**:
   Start by looking at the first few rows of the dataset to get a sense of its structure. Check the columns, data types, and a few sample entries. This helps to ensure that the data is loaded correctly and gives you an initial overview of the content.

2. **Check for Missing Values**:
   Identify if there are any missing values in the dataset.

3. **Distribution of Labels**:
   Examine the distribution of the target labels (classes).

4. **Text Data Characteristics (Bonus)**:
   Analyze the length of the text data. It is useful to calculate the number of words or characters in each text sample to understand how long the texts are. This will help you set a suitable `max_length` for tokenization and padding later. You can plot a histogram of text lengths to visualize the distribution.

5. **Common Words and Vocabulary (Bonus)**:
   Explore the most frequent words in the text data.

In [6]:
df.sample(10)

,text,label
2842,"Tunji, how's the queen? how are you doing. Thi...",ham
2370,"That day Ì_ say Ì_ cut ur hair at paragon, is ...",ham
4622,U need my presnts always bcz U cant mis love. ...,ham
3702,How is my boy? No sweet words left for me this...,ham
3797,"Feb &lt;#&gt; is \I LOVE U\"" day. Send dis t...",ham
795,it's really getting me down just hanging around.,ham
1146,Babe ? I lost you ... :-(,ham
1822,If you're thinking of lifting me one then no. ...,ham
2231,"K, wen ur free come to my home and also tel vi...",ham
3619,I meant as an apology from me for texting you ...,ham


In [13]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

# Data Cleaning and Preprocessing

In [7]:
def clean_text(text):
    text = str(text).lower()
    text = " ".join([contractions.fix(expanded_word) for expanded_word in text.split()])
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    return text

In [8]:
df['clean_text'] = df['text'].apply(clean_text)

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
def remove_stops(text):
  stop_words = r"\b(" + "|".join(stopwords) + r")\b"
  text = re.sub(stop_words, '', text).strip()
  return text


In [12]:
df['clean_text_1'] = df['clean_text'].apply(remove_stops)

In [13]:
df

,text,label,clean_text,clean_text_1
0,"Go until jurong point, crazy.. Available only ...",ham,go until jurong point crazy available only in ...,go jurong point crazy available bugis n gre...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif you oni,ok lar joking wif oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkt...
3,U dun say so early hor... U c already then say...,ham,you dun say so early hor you c already then say,dun say early hor c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah i do not think he goes to usf he lives aro...,nah think goes usf lives around though
...,...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam,this is the 2nd time we have tried 2 contact y...,2nd time tried 2 contact å£ pound prize...
5568,Will Ì_ b going to esplanade fr home?,ham,will ì b going to esplanade fr home,ì b going esplanade fr home
5569,"Pity, * was in mood for that. So...any other s...",ham,pity was in mood for that soany other suggest...,pity mood soany suggestions
5570,The guy did some bitching but I acted like i'd...,ham,the guy did some bitching but i acted like i w...,guy bitching acted like would interested...


In this section, we will focus on cleaning and filtering the dataset, preparing it for the text classification task. We will implement the following steps:

1. **Remove missing values**:  
   First, we eliminate any rows with missing values to ensure the dataset is complete and consistent.

2. **Filter by text length (Bonus)**:  
   To maintain a uniform dataset, we will filter the text samples by a specified word count range. This ensures that the texts are neither too short to lack context nor too long to introduce unnecessary complexity.

3. **English stopwords loading**:  
   We load a list of English stopwords to filter out commonly used but contextually insignificant words. This is an important step for improving the performance of the model, as stopwords do not contribute valuable information.

4. **Text cleaning**:  
   We apply a series of text cleaning steps to standardize and simplify the text data. This involves:
   
   - **Removing links (URLs)**:  
     Any URLs present in the text are removed as they are not meaningful for classification purposes.
   
   - **Removing special characters and punctuation**:  
     This step removes any non-alphabetical characters, ensuring the text only contains meaningful words.
   
   - **Lowercasing**:  
     All text is converted to lowercase for uniformity and to avoid case sensitivity issues.
   
   - **Removing English stopwords**:  
     Words that are part of the stopwords list are removed, as they do not add value to the classification task.
   
   - **Stemming or Lemmatization**:  
     Either stemming or lemmatization is applied to reduce words to their root or base form, ensuring consistency in word forms.

5. **Final cleanup**:
   Apply the cleanup function to the feature column.

By following these steps, the text will be cleaned, filtered, and ready for tokenization!

In [14]:
stemmer = SnowballStemmer('english', True)

In [15]:
def stem(text):
  tokens = nltk.word_tokenize(text)
  stemmer = SnowballStemmer("english", True)
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  text = " ".join(stemmed_tokens)
  return text

In [16]:
df['clean_2'] = df['clean_text_1'].apply(stem)

In [17]:
df = df.loc[:, ['clean_2', 'label']]
df

,clean_2,label
0,go jurong point crazi avail bugi n great world...,ham
1,ok lar joke wif oni,ham
2,free entri 2 wkli comp win fa cup final tkts m...,spam
3,dun say earli hor c alreadi say,ham
4,nah think goe usf live around though,ham
...,...,...
5567,2nd time tri 2 contact å£ pound prize 2 claim ...,spam
5568,ì b go esplanad fr home,ham
5569,piti mood soani suggest,ham
5570,guy bitch act like would interest buy someth e...,ham


In [19]:
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df[['label']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [59]:
text = "This is a sample sentence."
words = nltk.word_tokenize(text)
words

['This', 'is', 'a', 'sample', 'sentence', '.']

In [70]:
words = nltk.word_tokenize(str(X_train))

# Tokenization, Padding, and Data Splitting

In this step, we will prepare the text data for input into a model by converting the text into numerical sequences, padding them to a uniform length, and splitting the dataset into training and testing sets. Here's an overview of the steps involved:

1. **Tokenization**:
   We use a tokenizer to convert the cleaned text into numerical sequences. You can use `Tokenizer` tokenizer from `tensorflow.keras.preprocessing.text` package or any other tokenizer you like.

2. **Text to sequences**:
   After fitting the tokenizer on the cleaned text, we transform each text into a sequence of numbers, where each number corresponds to a token (word) in the text.

3. **Padding the sequences**:
   Since different texts may vary in length, we pad the sequences to ensure they all have the same length.

4. **Label encoding**:
   The labels (target values) also need to be converted into numerical form if they are not encoded.

5. **Train-test split**:
   The dataset is divided into training and testing sets. We allocate 80% of the data for training the model and reserve 20% for testing its performance.
   
   - The **training data** consists of the padded sequences used to train the model.
   - The **training labels** are the encoded labels corresponding to the training data.
   - The **testing data** is used to assess the model’s performance after training.
   - The **testing labels** are the encoded labels corresponding to the testing data.

6. **Data shape confirmation**:
   After splitting the data, we print the shape (dimensions) of both the training and testing sets to confirm that the data is properly divided and formatted.

By the end of this step, the text data will be transformed into padded numerical sequences, the labels will be encoded, and the data will be split into training and testing sets for model development and evaluation.

In [58]:
X = df['clean_2']
y = df[['label']]


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 10)

In [ ]:
values = X_train.values

In [76]:
vectorize_layer = TextVectorization(
    max_tokens=50000,
    output_mode='int',
    output_sequence_length = 100)

vectorize_layer.adapt(values)

# Building the Classifier

In this step, you will design and build a NLP Classifier model to classify text data. Below is a breakdown of the key components you'll implement, but it's up to you to decide how to configure them based on your understanding and experimentation:

1. **Model Type**:
   You will use a Sequential model, which allows you to stack layers in a linear sequence.

2. **Input Layer**:
   Define the shape of the input data. Consider the dimensions of your padded sequences and set the input shape accordingly.

3. **Embedding Layer**:
   The embedding layer will convert input tokens (integers) into dense vector representations. You will need to determine the size of the input dimension (based on your vocabulary) and the output dimension (embedding size).

4. **Bidirectional Simple RNN/LSTM Layers**:
   You can add one or more recurrent layers. Consider using Bidirectional layers to capture contextual information from both directions (forward and backward). You can chose SimpleRNN/GRU/LSTM to perform this step.

5. **Dense Layers**:
   Add one or more fully connected (Dense) layers to process the output from the RNN/GRU/LSTM layers.

6. **Output Layer**:
   The output layer should match the type of classification task you're working on. Consider using appropriate activation function with appropriate number of units.

7. **Model Summary**:
   After defining your model architecture, print a summary to review the number of layers, types of layers, and total parameters.

8. **Model Compilation**:
   Finally, compile the model by selecting an optimizer, a loss function, and metrics.

In [79]:
model = Sequential()
model.add(vectorize_layer)

model.add(Embedding(50000, 128))
model.add(Dropout(0.5))
model.add(GlobalAveragePooling1D())
model.add(Dense(1))

model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_3                 │ ?                           │     0 (unbuilt) │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [81]:
history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_split = .2)

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.8579 - loss: 0.3799 - val_accuracy: 0.8684 - val_loss: 0.3575
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 79ms/step - accuracy: 0.8684 - loss: 0.3511 - val_accuracy: 0.8684 - val_loss: 0.3462
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.8680 - loss: 0.3401 - val_accuracy: 0.8684 - val_loss: 0.3332
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.8608 - loss: 0.3367 - val_accuracy: 0.8684 - val_loss: 0.3106
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 93ms/step - accuracy: 0.8651 - loss: 0.2993 - val_accuracy: 0.8684 - val_loss: 0.2971
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.8686 - loss: 0.2667 - val_accuracy: 0.8684 - val_loss: 0.2651
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - accuracy: 0.8693 - loss: 0.2326 - val_accuracy: 0.8935 - val_loss: 0.2102
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - accuracy: 0.8989 - loss: 0.1851 - val

# Defining Batch Size, Creating Datasets, and Training the Model

In this step, you will define the batch size, create TensorFlow Datasets for both training and testing, and train the model. The key elements to consider are outlined below, and it is up to you to choose the specific configurations based on your preferences and experimentation:

1. **Batch Size**:
   Select a batch size for training and testing. The batch size determines how many samples will be processed together in one forward and backward pass during training.

2. **Creating Datasets**:
   Use TensorFlow’s `Dataset.from_tensor_slices()` to create datasets from the training and testing data.

3. **Batching the Datasets**:
   Batch the datasets by grouping the data into batches of the specified size.

4. **Training the Model**:
   Train the model by fitting it on the training dataset for a specified number of epochs. You will also need to provide the validation data to monitor the model’s performance on unseen data during training.

5. **Tracking Training History**:
   During training, the model’s performance metrics (such as loss and accuracy) will be tracked over the epochs, and the results will be stored in the `history` object.

# Model Evaluation

Once the model is trained, the next step is to evaluate its performance on the testing dataset.

1. **Evaluate the Model**:
   You will use the `evaluate()` method to assess the model’s performance on the test dataset.

2. **Testing Dataset**:
   Ensure that the testing dataset is properly prepared and batched, just like the training dataset.

4. **Loss Curve**:
   A loss curve plots the loss values for both the training and validation datasets over the epochs.

In [92]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss; {round(loss*100, ndigits=2)}")
print(f"Accuracy; {round(accuracy*100, ndigits=2)}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9213 - loss: 0.1430
Loss; 14.88
Accuracy; 91.82


# Model Inference

In this step, you will use the trained model to make predictions on new, unseen data (inference). Here’s an outline of the key points:

1. **Create Test Sample**:
   Create a string to test your modelm the goal here is to give the model Before making predictions, ensure that the new data is preprocessed in the same way as the training data. This includes tokenization, padding, and any other transformations you applied during the data preprocessing step. The data can be single text to see the result of the prediction.

2. **Model Prediction**:
   Use the `predict()` method to feed new samples into the trained model and obtain predictions. The model will output probabilities or predicted class labels based on the type of classification task (binary or multi-class).

3. **Interpreting Predictions**:
   The model will return probabilities for each class.

In [95]:
df.sample(10)

,clean_2,label
1373,bear pic nick tom pete dick fact type tri gay ...,1
884,yoyyooo know chang permiss drive mac usb flash...,0
276,tell rob mack gf theater,0
3184,mayb woke fuck 3 would problem,0
1849,got job wiproyou get everi thing life 2 3 year,0
2642,tri get lost fact tee hee,0
841,pls send comprehens mail pay much,0
4391,new year plan,0
959,,0
5353,guai ìï shd haf seen naughti ìï free today go jog,0


In [101]:
X_test

,clean_2
4635,k k pa lunch aha
2279,sorri call later meet
4545,never tri alon take weight tear come heart fal...
5084,hey happi birthday
5298,ill hand phone chat wit
...,...
1735,tire special
3745,lip sync shangela
96,watch telugu moviewat
510,8 latest gs still scroung ammo want give new a...


In [102]:
predict_history = model.predict(X_test)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


# Notebook Question:
- How did you handle text preprocessing? Why did you choose this approach?

- Why did you choose this model design?

- Why did you pick this number of layers or units for the model?

- Why did you select these evaluation methods?

- Does your model show signs of overfitting or underfitting? How do you know?

- What changes could you make to improve the model and fix overfitting or underfitting?

Answer Here: